# This file is example for using Reptile meta-learning algorithm for EEG-data


In [1]:
from braindecode.datasets.moabb import MOABBDataset
import os
import numpy as np
from braindecode.datautil.preprocess import (exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.datautil.windowers import create_windows_from_events
import pandas as pd
import numpy as np
import torch
import Meta_EEG as me

## Data loading from Braindecode (BCI IV 2a dataset)

In [ ]:
subject_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dataset_name = 'BCI_IV_2a'

In [13]:
ds = MOABBDataset(dataset_name = "BNCI2014001", subject_ids = subject_ids)
low_cut = 4.
high_cut = 38.
factor_new = 1e-3
init_block_size = 1000
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),
    Preprocessor(lambda x: x*1e6),
    Preprocessor('filter', l_freq=low_cut, h_freq=high_cut),
    Preprocessor(exponential_moving_standardize, factor_new=factor_new, init_block_size=init_block_size)
]
preprocess(ds, preprocessors)
sfreq = ds.datasets[0].raw.info['sfreq']
assert  all([d.raw.info['sfreq'] == sfreq for d in ds.datasets])
trial_start_ofset_samples = 0
w_ds = create_windows_from_events(
    ds,
    trial_start_offset_samples=trial_start_ofset_samples,
    trial_stop_offset_samples=0,
    preload=True
)
dataset_name = me.data_from_windows_dataset(dataset=w_ds, dataset_name=dataset_name, subjects=subject_ids, description='BCI IV 2a with 0.2 test and filtration 4-38 + exponential_m_sd (only EEG channels)')

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
4

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = me.inEEG_Net(num_classes=4, dropout1=0.52, dropout2=0.36, f1=16, sampling_rate=250, num_channels=62,
                     depth_multiplier=6, time_of_interest=500, time_points=1000, lowpass=50, point_reducer=5)
model.to(device)
params = me.meta_params()

28
28
28


In [20]:
path = "data_" + str(dataset_name) + "/"
data = pd.read_pickle(path + 'subj_1/' +"train_data.pkl")
print(len(data['X'][0][0]))

1000


In [74]:
type(w_ds)

TypeError: '<' not supported between instances of 'str' and 'int'